In [1]:
# %pip install scikit-learn

In [2]:
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib widget
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [3]:
df = pd.read_excel('../data/03-20-23.xlsx')
df['epsilon'] = df['epsilon']*1000

In [4]:
df.head()

,epsilon,perceived_epsilon,delta,interval,alpha,max_epoch,max_offset_size,mean_offset_size,percentile90_offset_size,percentile95_offset_size,percentile99_offset_size,max_counter,max_counter_size,mean_counter_size
0,2000,1,1000,1000,5,33948,4,1.059928,1,2,3.0,4,1,1.0
1,2000,1,1000,1000,10,18024,5,1.133866,2,2,4.0,5,1,1.0
2,2000,1,1000,1000,15,12176,6,1.210580,2,2,5.0,6,1,1.0
3,2000,1,1000,1000,20,8866,6,1.286812,2,2,5.0,6,1,1.0
4,2000,1,1000,2000,5,16308,5,1.284831,2,2,5.0,6,1,1.0


In [5]:
X = df[['epsilon', 'delta', 'interval', 'alpha']]
y = df['mean_offset_size']

# Split Data

In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

x_train shape: (4927, 4)
y_train shape: (4927,)
x_test shape: (1232, 4)
y_test shape: (1232,)


# Linear Regression

In [6]:
model = LinearRegression().fit(x_train, y_train)

In [7]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score, KFold

# define the number of folds for cross-validation
n_folds = 5

# define the cross-validation method
kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(x_train)

# perform cross-validation
cv_scores = cross_val_score(model, x_train, y_train, cv=kf, scoring='r2')

# calculate the mean and standard deviation of the cross-validation scores
mean_cv_score = cv_scores.mean()
std_cv_score = cv_scores.std()

# fit the model to the entire dataset
model.fit(x_train, y_train)

# calculate the R-squared value and RMSE on the entire dataset
y_pred = model.predict(x_test)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# print the results
print(f"Cross-validation R-squared score: {mean_cv_score:.3f} +/- {std_cv_score:.3f}")
print(f"R-squared on entire dataset: {r2:.3f}")
print(f"RMSE on entire dataset: {rmse:.3f}")

Cross-validation R-squared score: 0.860 +/- 0.010
R-squared on entire dataset: 0.859
RMSE on entire dataset: 3.456


In [8]:
slope = model.coef_[0]
intercept = model.intercept_
func = lambda x: slope * x + intercept
print(model.coef_)

[ 5.47606345e-04 -1.05479499e-04  1.30343264e-03  6.64748616e-01]


# Kernel Ridge

In [23]:
from sklearn.kernel_ridge import KernelRidge

In [30]:
model = KernelRidge(alpha = 1.0, kernel='rbf')

In [31]:
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [32]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
mse

459.59509120433927

# Support Vector Regression

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

In [34]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [36]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 1, 10],
    'epsilon': [0.01, 0.1, 1, 10],
}

In [37]:
svr = SVR(kernel='rbf')
grid_search = GridSearchCV(svr, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'epsilon': [0.01, 0.1, 1, 10],
                         'gamma': ['scale', 'auto', 0.1, 1, 10]})

In [38]:
svr = SVR(kernel='rbf', C=grid_search.best_params_['C'], gamma=grid_search.best_params_['gamma'], epsilon=grid_search.best_params_['epsilon'])
svr.fit(X_train, y_train)

SVR(C=100, epsilon=0.01, gamma=1)

In [39]:
y_pred = svr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

Mean Squared Error: 0.006680729930045099
R-squared Score: 0.9999213773628839


In [108]:
from sklearn.metrics.pairwise import rbf_kernel
import numpy as np

def svr_equation(svr_model):
    support_vectors = svr_model.support_vectors_
    dual_coefficients = svr_model.dual_coef_
    intercept = svr_model.intercept_
    gamma = svr_model._gamma

    def svr_function(x):
        kernel = rbf_kernel(support_vectors, x.reshape(1,-1), gamma=gamma)
        return np.dot(dual_coefficients[0], kernel) + intercept

    input_features = [f"x{i}" for i in range(support_vectors.shape[1])]
    equation = "y = " + str(intercept[0])

    for i in range(len(dual_coefficients[0])):
        if dual_coefficients[0][i] != 0:
            equation += f" + ({dual_coefficients[0][i]}) * rbf_kernel([{support_vectors[i]}], [{input_features}], gamma={gamma})"
    
    return equation

In [109]:
equation = svr_equation(svr)

print(equation)

y = 13.918486233210812 + (-32.62075662740013) * rbf_kernel([[-1.91282076 -0.85621153 -0.26685041 -1.34194581]], [['x0', 'x1', 'x2', 'x3']], gamma=1) + (-100.0) * rbf_kernel([[ 0.07094065 -1.24911894 -1.05266524 -0.4474605 ]], [['x0', 'x1', 'x2', 'x3']], gamma=1) + (-38.68907760589775) * rbf_kernel([[-0.92094006 -1.44557265 -0.07039671  1.34151011]], [['x0', 'x1', 'x2', 'x3']], gamma=1) + (16.829559136948994) * rbf_kernel([[ 1.06282135 -0.65975783  2.09059406 -1.34194581]], [['x0', 'x1', 'x2', 'x3']], gamma=1) + (3.0378112507845327) * rbf_kernel([[-2.90470147 -1.24911894 -0.85621153  1.34151011]], [['x0', 'x1', 'x2', 'x3']], gamma=1) + (100.0) * rbf_kernel([[ 0.07094065 -0.85621153  1.30477923 -1.34194581]], [['x0', 'x1', 'x2', 'x3']], gamma=1) + (-100.0) * rbf_kernel([[ 1.06282135  0.3225107   1.10832553 -0.4474605 ]], [['x0', 'x1', 'x2', 'x3']], gamma=1) + (67.44159404826465) * rbf_kernel([[ 0.07094065  1.10832553 -0.65975783 -0.4474605 ]], [['x0', 'x1', 'x2', 'x3']], gamma=1) + (-100

# Naive SVR

In [126]:
# train a linear regression model
model = SVR().fit(x_train, y_train)

In [127]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score, KFold

# define the number of folds for cross-validation
n_folds = 5

# define the cross-validation method
kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(x_train)

# perform cross-validation
cv_scores = cross_val_score(model, x_train, y_train, cv=kf, scoring='r2')

# calculate the mean and standard deviation of the cross-validation scores
mean_cv_score = cv_scores.mean()
std_cv_score = cv_scores.std()

# fit the model to the entire dataset
model.fit(x_train, y_train)

# calculate the R-squared value and RMSE on the entire dataset
y_pred = model.predict(x_test)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# print the results
print(f"Cross-validation R-squared score: {mean_cv_score:.3f} +/- {std_cv_score:.3f}")
print(f"R-squared on entire dataset: {r2:.3f}")
print(f"RMSE on entire dataset: {rmse:.3f}")

Cross-validation R-squared score: 0.775 +/- 0.022
R-squared on entire dataset: 0.784
RMSE on entire dataset: 4.286


In [130]:
# Get the support vectors, dual coefficients, and gamma value
support_vectors = svr.support_vectors_
dual_coefficients = svr.dual_coef_[0]
gamma = svr._gamma

# Get the coefficients for each input parameter
coefficients = np.zeros((4,))
for i in range(len(support_vectors)):
    kernel = rbf_kernel(support_vectors[i, np.newaxis], X, gamma=gamma)
    coefficients += dual_coefficients[i] * kernel.flatten()

# Add the intercept
intercept = np.mean(y) - np.dot(coefficients, np.mean(X, axis=0))

# Print the equation
equation_parts = []
for i, c in enumerate(coefficients):
    if c != 0:
        equation_parts.append("{:.2f} * X{}".format(c, i+1))
if intercept > 0:
    equation_parts.append("{:.2f}".format(intercept))
elif intercept < 0:
    equation_parts.append("- {:.2f}".format(abs(intercept)))
equation = " + ".join(equation_parts)
print("SVR equation: y = " + equation)


ValueError: operands could not be broadcast together with shapes (4,) (6159,) (4,) 

# Neural Network  

In [13]:
from sklearn.neural_network import MLPRegressor

features = ['epsilon', 'delta', 'interval', 'alpha']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# create a neural network model
model = MLPRegressor(hidden_layer_sizes=(100,50,20), activation='relu', solver='adam', max_iter=5000, random_state=42)

# fit the model on the training data
model.fit(X_train, y_train)



MLPRegressor(hidden_layer_sizes=(100, 50, 20), max_iter=5000, random_state=42)

In [14]:
# get the coefficients of the model
coefficients = model.coefs_

# get the intercepts of the model
intercepts = model.intercepts_

# initialize the equation_parts list
equation_parts = []

# loop over the coefficients for each hidden unit
for i, c in enumerate(coefficients[0]):
    
    # initialize a list of coefficient parts
    coefficient_parts = []
    
    # loop over the unique features
    for j, feature in enumerate(features):
        
        # append the coefficient for this feature
        coefficient_parts.append("{:.2f} * {}".format(c[j], feature))
    
    # append the intercept to the coefficient parts
    coefficient_parts.append("{:.2f}".format(intercepts[0][i]))
    
    # join the coefficient parts into a string and append to the equation_parts list
    equation_parts.append(" + ".join(coefficient_parts))

# join the equation parts into a single string with "+" separators
equation = " + ".join(equation_parts)

# print the equation
print("Neural network equation: y = " + equation)


Neural network equation: y = -0.00 * epsilon + 0.21 * delta + 0.27 * interval + 0.05 * alpha + -0.11 + -0.25 * epsilon + 0.04 * delta + -0.14 * interval + -0.02 * alpha + 0.35 + 0.16 * epsilon + -0.09 * delta + 0.11 * interval + 0.22 * alpha + 0.02 + -0.23 * epsilon + 0.03 * delta + 0.07 * interval + 0.08 * alpha + 0.35


In [ ]:
'''
epsilon: -0.00 -0.25 + 0.16 -0.23 = -0.32
delta: 0.21 + 0.04 -0.09 + 0.03 = 0.19
interval: 0.27 -0.14 + 0.11 + 0.07 = 0.31
alpha: 0.05 -0.02 + 0.22 + 0.08 = 0.33
'''

# Curve Fitting

In [65]:
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error, r2_score

# Define the function to fit
def func(X, a, b, c, d, e):
    x1 = X[:, 0]
    x2 = X[:, 1]
    x3 = X[:, 2]
    x4 = X[:, 3]
    return a*x1**5 + b*x2**2 + c*x3 + d*x4 + e

# Load data into a numpy array
data = df
X = df[['epsilon', 'delta', 'interval', 'alpha']].values
y = df['max_offset_size'].values

# Fit the function to the data
popt, pcov = curve_fit(func, X, y)

# Compute the MSE of the fitted model
y_pred = func(X, *popt)
mse = mean_squared_error(y, y_pred)
print('MSE:', mse)

# Compute the R2 score of the fitted model
r2 = r2_score(y, y_pred)
print('R2 score:', r2)

print('Function: {:.8f}*x1^2 + {:.8f}*x2^3 + {:.8f}*x3 + {:.8f}*x4 + {:.8f}'.format(*popt))

MSE: 22.418801975914853
R2 score: 0.5369317301670197
Function: 0.00000000*x1^2 + -0.00000000*x2^3 + 0.00089836*x3 + 0.35582557*x4 + 13.37918834


In [29]:
df

,epsilon,perceived_epsilon,delta,interval,alpha,max_epoch,max_offset_size,mean_offset_size,percentile90_offset_size,percentile95_offset_size,percentile99_offset_size,max_counter,max_counter_size,mean_counter_size
0,2000,1,1000,1000,5,33948,4,1.059928,1,2,3.0,4,1,1.0
1,2000,1,1000,1000,10,18024,5,1.133866,2,2,4.0,5,1,1.0
2,2000,1,1000,1000,15,12176,6,1.210580,2,2,5.0,6,1,1.0
3,2000,1,1000,1000,20,8866,6,1.286812,2,2,5.0,6,1,1.0
4,2000,1,1000,2000,5,16308,5,1.284831,2,2,5.0,6,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6154,20000,19,20000,19000,20,503,31,30.819068,31,31,31.0,45,1,1.0
6155,20000,19,20000,20000,5,1775,31,26.255958,29,29,31.0,32,1,1.0
6156,20000,19,20000,20000,10,861,31,30.139354,31,31,31.0,37,1,1.0
6157,20000,19,20000,20000,15,629,31,30.727929,31,31,31.0,42,1,1.0
